# 7 Linear Regression with TensorFlow using the California Housing Dataset
### The goal of this exercise is to implement a linear regression model using TensorFlow to predict house prices based on the California Housing Dataset. The dataset contains various features such as average income, housing average age, and more. Your task is to build a linear regression model and evaluate its performance.

## Import the required libraries:

In [1]:
import tensorflow as tf
import pandas as pd
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
raw = fetch_california_housing()
X = pd.DataFrame(data=raw['data'], columns=raw['feature_names'])
print(X.head())
y = pd.Series(raw['target'])

   MedInc  HouseAge  AveRooms  AveBedrms  Population  AveOccup  Latitude  \
0  8.3252      41.0  6.984127   1.023810       322.0  2.555556     37.88   
1  8.3014      21.0  6.238137   0.971880      2401.0  2.109842     37.86   
2  7.2574      52.0  8.288136   1.073446       496.0  2.802260     37.85   
3  5.6431      52.0  5.817352   1.073059       558.0  2.547945     37.85   
4  3.8462      52.0  6.281853   1.081081       565.0  2.181467     37.85   

   Longitude  
0    -122.23  
1    -122.22  
2    -122.24  
3    -122.25  
4    -122.25  


## Preprocess the data:

### -Normalize the features using the mean and standard deviation.
### -Split the dataset into training and testing sets (e.g., 80% for training, 20% for testing).

In [3]:
scaler = StandardScaler()
X = scaler.fit_transform(X)
# Note that X is now an ndarray
print(X[:5, :])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

[[ 2.34476576  0.98214266  0.62855945 -0.15375759 -0.9744286  -0.04959654
   1.05254828 -1.32783522]
 [ 2.33223796 -0.60701891  0.32704136 -0.26333577  0.86143887 -0.09251223
   1.04318455 -1.32284391]
 [ 1.7826994   1.85618152  1.15562047 -0.04901636 -0.82077735 -0.02584253
   1.03850269 -1.33282653]
 [ 0.93296751  1.85618152  0.15696608 -0.04983292 -0.76602806 -0.0503293
   1.03850269 -1.33781784]
 [-0.012881    1.85618152  0.3447108  -0.03290586 -0.75984669 -0.08561576
   1.03850269 -1.33781784]]


## Define the TensorFlow graph:

### -Create placeholders for the input features (X) and target variable (y).
### -Create variables for the model's weights (W) and bias (b).
### -Define the linear regression model using the equation: y_pred = X * W + b.
### -Define the loss function as the mean squared error between the predicted values and the true values.
### -Choose an optimizer (e.g., Gradient Descent) to minimize the loss function.

In [4]:
tf.compat.v1.disable_eager_execution()
X_placeholder = tf.compat.v1.placeholder(tf.float32, shape=[None, X_train.shape[1]])
y_placeholder = tf.compat.v1.placeholder(tf.float32, shape=[None])

W = tf.Variable(tf.compat.v1.random_normal([X_train.shape[1], 1]))
b = tf.Variable(tf.compat.v1.random_normal([1]))

y_pred = tf.matmul(X_placeholder, W) + b

loss = tf.reduce_mean(tf.square(y_pred - y_placeholder))

optimizer = tf.compat.v1.train.GradientDescentOptimizer(learning_rate=0.01)
train_op = optimizer.minimize(loss)

## Train the model:

### -Initialize TensorFlow session.
### -Initialize the model's variables.
### -Set the number of training epochs and the learning rate.
### -For each epoch, iterate through the training dataset and update the model's parameters using the optimizer.
### -Print the training loss at regular intervals.

In [15]:
batch_size = 32
num_epochs = 5

with tf.compat.v1.Session() as tfsession:
    tfsession.run(tf.compat.v1.global_variables_initializer())

    for epoch in range(num_epochs):
        for i in range(0, X_train.shape[0], batch_size):
            batch_X = X_train[i:i+batch_size]
            batch_y = y_train[i:i+batch_size]
            tfsession.run(train_op, feed_dict={X_placeholder: batch_X, y_placeholder: batch_y})

        if epoch % 100 == 0:
            train_loss = tfsession.run(loss, feed_dict={X_placeholder: X_train, y_placeholder: y_train})
            print(f"Epoch {epoch}: Training Loss = {train_loss}")

Epoch 0: Training Loss = 9.003902435302734


## Evaluate the model:

### -Use the trained model to make predictions on the test dataset.
### -Calculate the mean squared error (MSE) between the predicted and true values.
### -Print the MSE as a measure of the model's performance.

In [36]:
with tf.compat.v1.Session() as tfsession:
    tfsession.run(tf.compat.v1.global_variables_initializer())

    for epoch in range(num_epochs):
        for i in range(0, X_train.shape[0], batch_size):
            batch_X = X_train[i:i+batch_size]
            batch_y = y_train[i:i+batch_size]
            tfsession.run(train_op, feed_dict={X_placeholder: batch_X, y_placeholder: batch_y})

        if epoch % 100 == 0:
            train_loss = tfsession.run(loss, feed_dict={X_placeholder: X_train, y_placeholder: y_train})
            print(f"Epoch {epoch}: Training Loss = {train_loss}")

      # Hacer predicciones en el conjunto de prueba
    y_pred_test = tfsession.run(y_pred, feed_dict={X_placeholder: X_test})

    y_test_probar = y_test.values
      # Calcular el MSE
    mse = tf.reduce_mean(tf.square(y_pred_test - y_test_probar))

      # Obtener el valor del MSE
    mse_value = tfsession.run(mse)

      # Imprimir el MSE
    print("Mean Squared Error (MSE):", mse_value)

    print(y_pred_test)
    print(y_test_probar)




Epoch 0: Training Loss = 1.402226209640503
Mean Squared Error (MSE): 1.3285310260459566
[[2.2013977]
 [2.0855694]
 [1.9790822]
 ...
 [2.0120294]
 [2.0914078]
 [2.0508668]]
[3.55  0.707 2.294 ... 3.5   2.273 1.417]
